In [ ]:
!pip install -e ../../


In [1]:
from terratorch.tasks import ObjectDetectionTask
from terratorch.datamodules.xview import XviewDataModule
from lightning.pytorch import Trainer
from torchvision import transforms
import logging


/dccstor/terratorch/users/rkie/gitco/terratorch/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!wget https://raw.githubusercontent.com/akihironitta/xView-PyTorch/refs/heads/master/datasets.py
#!mv datasets.py ../../terratorch/datasets/xview.py


--2025-08-14 07:38:10--  https://raw.githubusercontent.com/akihironitta/xView-PyTorch/refs/heads/master/datasets.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5336 (5.2K) [text/plain]
Saving to: ‘datasets.py.2’

datasets.py.2       100%[===================>]   5.21K  --.-KB/s    in 0.001s  

2025-08-14 07:38:10 (4.03 MB/s) - ‘datasets.py.2’ saved [5336/5336]



In [ ]:
img_transform = transforms.Compose([
    transforms.Resize((512, 512)), 
    transforms.ToTensor()
])

dm = XviewDataModule(data_dir = '/dccstor/terratorch/users/rkie/data/xview/train_images', ann_file = '/dccstor/terratorch/users/rkie/data/xview/xView_train.geojson')

In [4]:
model_args = {
      'framework': 'mask-rcnn',
      'backbone': 'prithvi_eo_v2_300',
      'num_classes': 84,
      'backbone_pretrained': True,
      'framework_min_size': 224,
      'framework_max_size': 224,
      'backbone_bands': ['BLUE','GREEN','RED'],
}
m = ObjectDetectionTask(model_factory='ObjectDetectionModelFactory', model_args=model_args)

In [5]:
trainer = Trainer(
    devices=1,
    precision="16-mixed",
    max_epochs=1,
    default_root_dir='output',
    log_every_n_steps=1,
    check_val_every_n_epoch=1
)

Using bfloat16 Automatic Mixed Precision (AMP)
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [6]:
trainer.fit(model=m, datamodule=dm)  


  | Name          | Type                 | Params | Mode 
---------------------------------------------------------------
0 | model         | ObjectDetectionModel | 343 M  | train
1 | train_metrics | MetricCollection     | 0      | train
2 | val_metrics   | MetricCollection     | 0      | train
3 | test_metrics  | MetricCollection     | 0      | train
---------------------------------------------------------------
343 M     Trainable params
0         Non-trainable params
343 M     Total params
1,375.256 Total estimated model params size (MB)
603       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/dccstor/terratorch/users/rkie/gitco/terratorch/.venv/lib/python3.12/site-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/dccstor/terratorch/users/rkie/gitco/terratorch/.venv/lib/python3.12/site-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/dccstor/terratorch/users/rkie/gitco/terratorch/.venv/lib/python3.12/site-packages/torch/utils/data/_utils/collate.py", line 398, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/dccstor/terratorch/users/rkie/gitco/terratorch/.venv/lib/python3.12/site-packages/torch/utils/data/_utils/collate.py", line 212, in